In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# drop NAN values
df = df.dropna()

In [5]:
# Get the independent features
X = df.drop('label', axis=1)

In [6]:
# Get dependent features
y = df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.5.0'

In [13]:
# Import necessary libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout

In [14]:
voc_size = 5000

# Onehot Representation

In [15]:
messages = X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re  # regular expression
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.stem.porter import PorterStemmer

In [21]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # substituting everything with blank space apart from a-zA-Z.
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[562, 4088, 1388, 3739, 587, 4525, 2100, 122, 283, 573],
 [3142, 2394, 4839, 1174, 2983, 835, 2185],
 [4255, 1817, 2210, 2954],
 [1731, 2220, 959, 430, 3775, 34],
 [274, 2983, 3859, 4639, 1469, 966, 2983, 399, 280, 447],
 [2550,
  1920,
  1809,
  3054,
  158,
  4355,
  1114,
  53,
  1789,
  2665,
  3724,
  2248,
  1444,
  1222,
  2185],
 [3678, 1685, 1346, 1359, 4268, 19, 1442, 4164, 608, 482, 3255],
 [1704, 3289, 955, 1606, 1865, 2315, 4355, 3418, 608, 482, 3255],
 [4070, 1005, 2085, 4899, 3082, 1040, 4528, 3504, 4355, 3242],
 [1857, 3478, 4381, 3457, 1747, 3749, 416, 1423],
 [4897, 58, 3480, 4847, 3842, 4878, 1514, 4470, 4921, 3881, 1758],
 [430, 4716, 587, 1040, 4355, 1865],
 [1920, 3857, 4187, 4410, 3743, 1400, 2092, 1131, 1752],
 [4538, 780, 2526, 878, 2228, 727, 4907, 608, 482, 3255],
 [1439, 3993, 3109, 4449, 3892, 608, 482, 3255],
 [4724, 2944, 4400, 3638, 3721, 3673, 1788, 1002, 4917, 3104],
 [572, 2119, 2394],
 [3288, 1822, 3247, 1766, 4355, 2130, 881, 2185],
 [3478, 69, 483

# Embedding Representation

In [24]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  122  283  573]
 [   0    0    0 ... 2983  835 2185]
 [   0    0    0 ... 1817 2210 2954]
 ...
 [   0    0    0 ...  608  482 3255]
 [   0    0    0 ... 4211 1014 1869]
 [   0    0    0 ... 4026 3983 4205]]


In [25]:
len(embedded_docs)

18285

In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  562,
       4088, 1388, 3739,  587, 4525, 2100,  122,  283,  573])

# Creating Model

In [44]:
embedding_vec_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vec_features, input_length=sent_length))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [46]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [48]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 38ms/step - loss: 0.3159 - accuracy: 0.8504 - val_loss: 0.1992 - val_accuracy: 0.9150
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1560 - accuracy: 0.9396 - val_loss: 0.2045 - val_accuracy: 0.9206
Epoch 3/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1178 - accuracy: 0.9562 - val_loss: 0.2306 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0940 - accuracy: 0.9648 - val_loss: 0.2548 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0800 - accuracy: 0.9720 - val_loss: 0.2407 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0711 - accuracy: 0.9768 - val_loss: 0.2467 - val_accuracy: 0.9150
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0564 - accuracy: 0.9816 - val_loss: 0.2652 - val_accuracy: 0.913

# Performance Metrics and Accuracy

In [49]:
y_pred = model.predict_classes(X_test)

C:\Users\Shree\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [34]:
from sklearn.metrics import confusion_matrix

In [50]:
confusion_matrix(y_test, y_pred)

array([[3125,  294],
       [ 230, 2386]], dtype=int64)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9131731565865783

In [52]:
model.save('bimodel.h5')

In [38]:
from tensorflow.keras.models import load_model

In [53]:
model1 = load_model('bimodel.h5')

In [54]:
x = ['Al-Qaida could strike US from Afghanistan base next year, spies say']

In [55]:
ohr = [one_hot(words, voc_size) for words in x]
ohr

[[979, 530, 3514, 976, 430, 450, 3859, 1514, 542, 2759, 735, 2292]]

In [56]:
x_pro = pad_sequences(ohr, padding='pre', maxlen=sent_length)
x_pro

array([[   0,    0,    0,    0,    0,    0,    0,    0,  979,  530, 3514,
         976,  430,  450, 3859, 1514,  542, 2759,  735, 2292]])

In [57]:
model1.predict(x_pro)

array([[0.9905632]], dtype=float32)